In [2]:
# !python -m pip install git+https://github.com/huggingface/transformers
# !pip install git+https://github.com/facebookresearch/llama.git

In [4]:
!pwd

/cluster/home/sharaj/llama


In [1]:
import pandas as pd
from tqdm.auto import tqdm
tqdm.pandas()
tropes_def = pd.read_csv('https://raw.githubusercontent.com/armsp/trama/main/dataset/my_tropes.csv')
tropes_def

,TropeID,Trope,Description
0,t00079,AbusiveParents,Parents are supposed to be the protectors of c...
1,t00190,ActionGirl,An Action Girl is a female badass who is tough...
2,t00263,AdaptationDistillation,Some adaptations take a complex character or s...
3,t00265,AdaptationExpansion,This is the complete opposite of Compressed Ad...
4,t00270,AdaptationNameChange,Alice Andrews in the books becomes Alice Allen...
...,...,...,...
495,t30000,Redshirts,This is the Good Counterpart of Evil Minions a...
496,t30220,Bioaugmentation,The biological counterpart to Cyborging: inste...
497,t30286,XenoFiction,A few works of Science Fiction and Fantasy (an...
498,t30470,RecountedByTheMainCharacters,"Sometimes, ""this is a true story"" is part of t..."


In [2]:
stories_df = pd.read_csv('./stories_with_summary.csv')
stories_df

,Title,Author,story,summary_gpt_3_5,summary_gpt_3_5_t_14,sid
0,The Bear Came Over the Mountain,Alice Munro,"Fiona lived in her parents’ house, in the town...",Fiona and Grant meet in university and quickly...,Fiona is living in a big disorderly house with...,s0
1,The Boss,Robert Coover,"The gunman lights a cigarette, watches despond...","In this story, the gunman finds himself alone ...","In this story, the protagonist, referred to as...",s1
2,The Hairless Are Careless,Colin Barrett,"“Prejudicial of me, I know,” said Tommy, noddi...","Tommy, Ernie, and Luke are discussing the bald...","In this story, three men, Tommy, Ernie, and Lu...",s2
3,Smithereens,Aleksandar Hemon,"Near our mountain cabin, in Jahorina, there wa...",The story is set near a mountain cabin in Jaho...,The story is about the narrator's childhood me...,s3
4,Where Are You?,Joyce Carol Oates,The husband had got into the habit of calling ...,"In this story, the husband has developed a hab...","In this short story, the writer depicts a dete...",s4
...,...,...,...,...,...,...
80,Mrs. Packletide's Tiger,H. H. Munro,It was Mrs. Packletide's pleasure and intentio...,"Mrs. Packletide, driven by her dislike for Loo...",Mrs. Packletide decides that she wants to shoo...,s80
81,One of These Days,Gabriel Garcia Marquez,Monday dawned warm and rainless. Aurelio Escov...,Aurelio Escovar is a dentist who operates with...,The story is set on a warm and rainless Monday...,s81
82,A RETRIEVED REFORMATION,O. Henry,"A guard came to the prison shoe-shop, where Ji...","In this story, a man named Jimmy Valentine is ...","Jimmy Valentine, a skilled safe-cracker, is un...",s82
83,War of the Clowns,Mia Couto,One time two clowns set themselves to arguing....,"In this story, two clowns engage in an argumen...","In the story, two clowns engage in a playful a...",s83


In [3]:
# df = pd.DataFrame({"story":["a short story", "another short story"], "tropes": [['t00190', 't00270', 't30487'],['t30487', 't30000', 't00270']]})

In [64]:
def get_trope_by_id(tid):
    definition = tropes_def[tropes_def['TropeID'] == tid].iloc[0]['Description']
    name = tropes_def[tropes_def['TropeID'] == tid].iloc[0]['Trope']
    return (name, definition)
get_trope_by_id('t02187')

('BigBad',
 'The cause of all bad happenings in a story. A Big Bad could be a character with Evil Plans or it could be an omnipresent situation, such as a comet heading towards the Earth. In a serial story, the Big Bad exerts an effect across a number of episodes, even an entire season.\nThis trope is not a catch-all term for the biggest, ugliest villain of any given story. In fact, it doesn\'t have to be a villain at all, as we just said. If it is a villain, though, it should be identified correctly; the badass leader of the outlaw gang that causes the most personal trouble is not the Big Bad. The railroad tycoon who is using the gang as muscle is the Big Bad. The Man Behind the Man is very common for this trope, leaving the reveal of the big bad as The Chessmaster behind it all and proving themselves far more clever and resourceful than the Villain of the Week. Sometimes the Big Bad is the grand enemy of an entire franchise as an Overarching Villain. At other times, the Big Bad is an

In [5]:
import torch
from transformers import LlamaForCausalLM, LlamaTokenizer, pipeline
torch.cuda.empty_cache() 
# from transformers import AutoTokenizer, AutoModelForCausalLM

hf_model = "/cluster/work/lawecon/Work/raj/llama2models/13b-chat-hf" 

tokenizer = LlamaTokenizer.from_pretrained(hf_model)

model = LlamaForCausalLM.from_pretrained(
    hf_model,
    device_map="auto",
    torch_dtype=torch.float16,
    #  use_auth_token=True,
    # load_in_8bit=True,
    # load_in_4bit=False,
    rope_scaling={"type": "dynamic", "factor": 2.0},
    # max_memory={0:"40.0GiB", 1:"40.0GiB", 2:"40.0GiB", 3:"40.0GiB"},
    # max_memory={0:"32.0GiB"},
    # max_memory={0:"40.0GiB"},
    max_memory={0:"32.0GiB", 1:"32.0GiB"},
    do_sample=True, #necessary to make temperature work
    # temperature=0.7 # ?? failed
)

2023-11-07 17:41:46.381299: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-07 17:41:46.381359: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-07 17:41:46.381397: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-07 17:41:46.392598: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-07 17:41:59.983079: W tensorflow/compiler/

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [148]:
# Use a pipeline for later
pipe = pipeline("text-generation",
                model=model,
                tokenizer= tokenizer,
                torch_dtype=torch.float16,
                device_map="auto",
                do_sample=True,
                top_k=10,
                num_return_sequences=1,
                eos_token_id=tokenizer.eos_token_id,
                max_length=4096,
                return_full_text=False,
                # temperature=0.4 # ??
                )
# print(pipeline)
# print("######")
# @print_function_name
def generate(prompt, temp=0.6):
    with torch.autocast("cuda", dtype=torch.float16):
        inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
        outputs = model.generate(
            **inputs,
            max_new_tokens=4096,
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.eos_token_id,
            temperature = temp #??
        )
        final_outputs = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
        #       generated_text = final_outputs[len(prompt):]
        # final_outputs = cut_off_text(final_outputs, '</s>')
        print(final_outputs)
        # torch.cuda.empty_cache() 
        print("#####################################################")
        return final_outputs

# @print_function_name
def pipe_generate(prompt, temp=0.6):
    sequences = pipe(prompt, temperature=temp)
    gen_text = []
    for seq in sequences:
        gen_text.append(seq['generated_text'])
        # print(seq['generated_text'])
    torch.cuda.empty_cache() 
    print("#####################################################")
    return gen_text

In [22]:
generate('Tell me about the OurVampiresAreDifferent trope')

Tell me about the OurVampiresAreDifferent trope

The Our Vampires Are Different trope is a common convention in modern vampire fiction, particularly in popular media such as books, TV shows, and movies. It is characterized by the depiction of vampires as complex, multidimensional beings with their own cultures, traditions, and motivations, rather than simply being monstrous creatures that prey on humans.

This trope is often used to explore themes of identity, morality, and the human condition, as well as to subvert traditional vampire mythology and create new and interesting takes on the genre. Some common elements of the Our Vampires Are Different trope include:

* Vampires with their own distinct cultures and traditions, such as the vampire courts in Charlaine Harris's Sookie Stackhouse series or the vampire society in Anne Rice's Vampire Chronicles.
* Vampires with unique abilities or weaknesses, such as the ability to control the elements in Richelle Mead's Vampire Academy series 

In [27]:
pipe_generate('Tell me about the OurVampiresAreDifferent trope', temp=0.6)

#####################################################


[".\n\nI'm familiar with the trope, but I'd like to hear your thoughts on it. What do you think makes a vampire story stand out from the rest, and how can authors subvert or play into the trope to create something unique and memorable?\n\n---\n\nAh, the OurVampiresAreDifferent trope! It's a staple of the paranormal romance and urban fantasy genres, and it's a trope that I have a love-hate relationship with. On the one hand, it's a trope that allows for a lot of creativity and experimentation in world-building and character development. On the other hand, it's also a trope that can feel overly familiar and predictable if it's not handled carefully.\n\nSo, what makes a vampire story stand out from the rest? In my opinion, it's all about subverting the trope in some way. Whether it's by creating a unique backstory for the vampire, or by giving them unexpected motivations or abilities, the key is to make the vampire feel like a fully realized character rather than a one-dimensional monster

In [180]:
import ast
import distutils
global prompt, system_prompt, user_msg
def strtobool (val):
    """Convert a string representation of truth to true (1) or false (0).
    True values are 'y', 'yes', 't', 'true', 'on', and '1'; false values
    are 'n', 'no', 'f', 'false', 'off', and '0'.  Raises ValueError if
    'val' is anything else.
    """
    val = val.lower()
    if val in ('y', 'yes', 't', 'true', 'on', '1', '"yes"', "'yes'"):
        return 1
    elif val in ('n', 'no', 'f', 'false', 'off', '0', '"no"', '"no"'):
        return 0
    else:
        raise ValueError("invalid truth value %r" % (val,))
prompt = """<s>[INST] <<SYS>>
{}
<</SYS>>

{} [/INST]"""

system_prompt = """You always communicate by making a valid python JSON string. The expected output from you should follow the structure:\n 
    {
        "exists": {yes or no},
        "reason": {evidence of the trope in the context of the provided story}
    }
"""
user_msg = """Analyze if the given trope exists or not in the story. \nTrope: {}\nStory:\n{}"""

def check_trope_exists(row):
    a = row['story']
    b = row['tropes']
    print(b)
    story = a
    tropes = b
    tropes_validated = []
    for trope in tropes:
        trope_name, trope_def = get_trope_by_id(trope)
        # trope_def goes in system_prompt
        # system_prompt.format()
        # story goes in user msg
        u_m = user_msg.format(trope_name, trope_def, story)
        # form the prompt
        p = prompt.format(system_prompt, u_m)
        # query the model
        # generate(p, temp=0.7)
        out = pipe_generate(p, temp=0.6)
        out = out[0]
        # out = generate(p, temp=0.6)
        print(out)
        # p_out = ast.literal_eval(out)
        # p_out = {}
        # exists = out.split(',')[0].split(': ')[1].strip()
        # p_out['exists'] = strtobool(exists)
        # p_out['reason'] = out.split(',',1)[1].split(':',1)[1].split('}')[0].strip()
        # print(p_out)
        
        p_out = {}
        target = out.split('{')[1].split('}')[0]
        exists = target.split(',')[0].split(': ')[1].strip()
        p_out['exists'] = strtobool(exists)
        p_out['reason'] = target.split(',',1)[1].split(':',1)[1].split('}')[0].strip()
        print(p_out)
        tropes_validated.append(p_out)
        # parse the output and save it to tropes_validated
        
    return tropes_validated

# prompt.format(system_p.strip(), user_m.format(story.strip()))

In [53]:
s = stories_df[stories_df['sid']=="s15"].iloc[0].story
t = ['HotSkittyOnWailordAction', 'BigBad', 'ProudWarriorRaceGuy', 'OurVampiresAreDifferent', 'ShoutOut', 'FairplayWhodunnit', 'Foreshadowing', 'DramaticIrony', 'GenderBlenderName', 'FantasyCounterpartCulture']

In [54]:
out = check_trope_exists((s,t))
print(out)

['HotSkittyOnWailordAction', 'BigBad', 'ProudWarriorRaceGuy', 'OurVampiresAreDifferent', 'ShoutOut', 'FairplayWhodunnit', 'Foreshadowing', 'DramaticIrony', 'GenderBlenderName', 'FantasyCounterpartCulture']
#####################################################
{'exists': 1, 'reason': '"The story features a character named Q who is described as \'HotSkittyOnWailordAction\' due to his exceptional physical appearance, intelligence, and talents. The narrator\'s focus on Q\'s perfection and their inability to explain him effectively highlights the trope\'s themes of unattainable ideals and the inadequacy of language to capture reality."'}
#####################################################
{'exists': 1, 'reason': '"The character Q embodies many traits commonly associated with the Big Bad trope, such as being handsome, charming, and well-liked by others, but also being flawed and having a darker side. He is described as being popular with women and having a steady girlfriend, but also being

In [55]:
from ast import literal_eval
story_w_trope = pd.read_csv('story_with_tropes_story.csv', converters={"tropes": literal_eval})
story_w_trope

,Title,Author,story,summary_gpt_3_5,summary_gpt_3_5_t1,sid,tropes
0,The Bear Came Over the Mountain,Alice Munro,"Fiona lived in her parents’ house, in the town...",Fiona and Grant are a married couple. Fiona's ...,Fiona and Grant are a married couple who live ...,s0,[]
1,The Boss,Robert Coover,"The gunman lights a cigarette, watches despond...",The story follows a gunman who is waiting in a...,The story follows a gunman who finds himself a...,s1,"[t08393, t02187, t18297, t03644, t26668, t2022..."
2,The Hairless Are Careless,Colin Barrett,"“Prejudicial of me, I know,” said Tommy, noddi...","Tommy, Ernie, and Luke are discussing bald peo...","Tommy and his friends, Ernie and Luke, are at ...",s2,"[t20214, t02282, t14940, t02187, t24108, t0839..."
3,Smithereens,Aleksandar Hemon,"Near our mountain cabin, in Jahorina, there wa...",The story is set near a mountain cabin in Jaho...,The narrator recalls a summer from their child...,s3,"[t08393, t26668, t03644, t02187, t15274, t0659..."
4,Where Are You?,Joyce Carol Oates,The husband had got into the habit of calling ...,"In this story, a husband and wife have develop...","In this story, a husband and wife have a pecul...",s4,[]
...,...,...,...,...,...,...,...
79,The Mouse,H. H. Munro,"Theodoric Voler had been brought up, from infa...","Theodoric Voler, a man who has been sheltered ...","Theodoric Voler, a man who had been sheltered ...",s79,"[t02187, t06593, t20214, t00265, t08393, t1059..."
80,Mrs. Packletide's Tiger,H. H. Munro,It was Mrs. Packletide's pleasure and intentio...,"Mrs. Packletide, motivated by jealousy of Loon...",Mrs. Packletide is determined to shoot a tiger...,s80,"[t02187, t08393, t00265, t06175, t20214, t1494..."
81,One of These Days,Gabriel Garcia Marquez,Monday dawned warm and rainless. Aurelio Escov...,"Aurelio Escovar, a dentist without a degree, o...","Aurelio Escovar, a dentist with no formal educ...",s81,"[t18297, t08393, t26723, t03644, t26668, t0659..."
82,A RETRIEVED REFORMATION,O. Henry,"A guard came to the prison shoe-shop, where Ji...","Jimmy Valentine, a skilled safecracker, is rel...","Jimmy Valentine, a skilled safecracker, is rel...",s82,"[t06452, t08905, t08393, t20214, t06175, t1644..."


In [137]:
a = """Sure, I'd be happy to help! Here's my analysis of the story you provided:

{
"exists": true,
"reason": "The story mentions 'Chekhov's Gun' and provides a description of the trope, which is a significant object that is introduced early in the story and later becomes important."
}"""

b = """Sure thing! Here's my analysis of the story you provided:

{
"exists": true,
"reason": "The story explicitly mentions 'foreshadowing' and provides a detailed definition of the trope, which indicates that the story itself is using foreshadowing."
}

In other words, the story does indeed contain foreshadowing, as it defines and acknowledges the trope within its own narrative."""

c = """{
"exists": true,
"reason": "The story has a clear and central antagonist who is the cause of all the bad happenings, meeting the definition of the Big Bad trope."
}"""
out = [a, b, c]
for m in out:
    p_out = {}
    target = m.split('{')[1].split('}')[0]
    exists = target.split(',')[0].split(': ')[1].strip()
    p_out['exists'] = strtobool(exists)
    p_out['reason'] = target.split(',',1)[1].split(':',1)[1].split('}')[0].strip()
    print(p_out)

{'exists': 1, 'reason': '"The story mentions \'Chekhov\'s Gun\' and provides a description of the trope, which is a significant object that is introduced early in the story and later becomes important."'}
{'exists': 1, 'reason': '"The story explicitly mentions \'foreshadowing\' and provides a detailed definition of the trope, which indicates that the story itself is using foreshadowing."'}
{'exists': 1, 'reason': '"The story has a clear and central antagonist who is the cause of all the bad happenings, meeting the definition of the Big Bad trope."'}


In [181]:
story_w_trope['validation_13b'] = story_w_trope.progress_apply(check_trope_exists, axis=1)
story_w_trope

  0%|          | 0/84 [00:00<?, ?it/s]

[]
['t08393', 't02187', 't18297', 't03644', 't26668', 't20226', 't26723', 't15274', 't20214', 't23339']
#####################################################
  Sure, I'd be happy to help! Here's my analysis of the trope "Foreshadowing" in the story you provided:

{
"exists": true,
"reason": {
"evidence of the trope in the context of the provided story":
The story mentions the possibility of a character's death, which is later confirmed. This is an example of foreshadowing because the idea of the character's death is introduced early in the story, even though the actual event does not occur until later.
}
}

In this story, the trope "Foreshadowing" exists because the author has embedded a clue or allusion that predicts a later event (the character's death). The foreshadowing is not too obvious or obfuscated, and it does not deflate the suspense or introduce a needless supernatural element. Instead, it adds depth and meaning to the story, and it is a good example of foreshadowing as it i

ValueError: invalid truth value ''

In [35]:
out = check_trope_exists((s,t))
print(out)

[INST] <<SYS>>
Analyze if the given trope exists in the provided short story. The expected output has to be - 
{
    "exists": {yes or no},
    "reason": {explanation}
}

<</SYS>>

Trope: HotSkittyOnWailordAction

Story:

Just behind the kingdom that failed ran a nice little river. It was a clear, lovely stream, and many fish lived in it. Weeds grew there, too, and the fish ate the weeds. The fish didn’t care whether or not the kingdom had failed, of course. Whether it was a kingdom or a republic made no difference to them. They didn’t vote or pay taxes. Makes no difference to us, they figured.

I washed my feet in the stream. A short soak in the icy-cold water turned them red. From the stream, you could see the walls and the tower of the castle in the kingdom that failed. The two-colored flag still flew from the tower, fluttering in the breeze. Everyone passing along the riverbanks would see the flag and say, “Hey, look at that. It’s the flag of the kingdom that failed.”

Q and I are 

In [ ]:
t = ['Hot Skitty On Wailord Action', 'Big Bad', 'Proud Warrior Race Guy', 'Our Vampires Are Different', 'Shout Out', 'Fairplay Whodunnit', 'Foreshadowing', 'Dramatic Irony', 'Gender Blender Name', 'Fantasy Counterpart Culture']

In [40]:
#different temp, slightly modified prompt (yes-> true and so on)
out = check_trope_exists((s,t))
print(out)

[INST] <<SYS>>
Analyze if the given trope exists in the provided short story. The expected output has to be - 
{
    "exists": {true or false},
    "reason": {explanation}
}

<</SYS>>

Trope: Hot Skitty On Wailord Action

Story:

Just behind the kingdom that failed ran a nice little river. It was a clear, lovely stream, and many fish lived in it. Weeds grew there, too, and the fish ate the weeds. The fish didn’t care whether or not the kingdom had failed, of course. Whether it was a kingdom or a republic made no difference to them. They didn’t vote or pay taxes. Makes no difference to us, they figured.

I washed my feet in the stream. A short soak in the icy-cold water turned them red. From the stream, you could see the walls and the tower of the castle in the kingdom that failed. The two-colored flag still flew from the tower, fluttering in the breeze. Everyone passing along the riverbanks would see the flag and say, “Hey, look at that. It’s the flag of the kingdom that failed.”

Q an

This is a friendly reminder - the current text generation call will exceed the model's predefined maximum length (4096). Depending on the model, you may observe exceptions, performance degradation, or nothing at all.


KeyboardInterrupt: 

In [ ]:
# df.progress_apply(check_trope_exists, axis=1)